### MANEJO DE DATOS FALTANTES: DETECCIÓN Y EXPLORACIÓN

#### IMPORTAMOS LAS LIBRERÍAS A EMPLEAR

In [1]:
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import session_info
import upsetplot

C:\Users\CRISTINA\anaconda3\envs\DataScience\lib\site-packages\upsetplot\plotting.py:20: MatplotlibDeprecationWarning: The matplotlib.tight_layout module was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  from matplotlib.tight_layout import get_renderer


#### IMPORTAR FUNCIONES PERSONALIZADAS

#### CONFIGURACIÓN DEL ASPECTO GENERAL DE LAS GRÁFICAS

In [2]:
%matplotlib inline

sns.set(rc = {
    "figure.figsize": (10, 10)})

sns.set_style("whitegrid")

#### OPERAMOS CON VALORES FALTANTES

In [3]:
#Visualización diversos casos de valores faltantes
print(
    None or True,
    None or False,
    None == None,
    None is None,
    type(None),
    sep = "\n"
)

True
False
True
True
<class 'NoneType'>


In [4]:
#Visualización diversos casos de valores faltantes con NUMPY
print(
    np.nan or True,
    np.nan == np.nan,
    np.nan is np.nan,
    np.nan/2,
    type(np.nan),     #Tipo de dato: float, podemos hacer operaciones
    np.isnan(np.nan), #Verificamos si es un valor faltante
    sep = "\n"
)

nan
False
True
nan
<class 'float'>
True


In [5]:
#Visualización diversos casos de valores faltantes con PANDAS
test_missing_df = pd.DataFrame.from_dict(
    data = dict(
        x = [0, 1, np.nan, np.nan, None],
        y = [0, 1, pd.NA, np.nan, None]
    )
)

#Vemos que PANDAS tiene distintas maneras de datos faltantes
test_missing_df

,x,y
0,0.0,0
1,1.0,1
2,NaN,<NA>
3,NaN,NaN
4,NaN,None


In [6]:
#Preguntamos si hay valores faltantes
test_missing_df.isna()

,x,y
0,False,False
1,False,False
2,True,True
3,True,True
4,True,True


In [7]:
#Preguntamos si hay valores nulos
test_missing_df.isnull()

,x,y
0,False,False
1,False,False
2,True,True
3,True,True
4,True,True


In [8]:
#Preguntamos para una sola columna
test_missing_df.x.isnull()

0    False
1    False
2     True
3     True
4     True
Name: x, dtype: bool

In [9]:
#Serie de datos numéricos
pd.Series([1, np.nan])

0    1.0
1    NaN
dtype: float64

In [10]:
#Serie de datos temporales
pd.Series([pd.to_datetime("2022-01-01"), np.nan])

0   2022-01-01
1          NaT
dtype: datetime64[ns]

In [11]:
#Serie extra (en algunos casos -1 es un valor faltante, pero Pandas NO lo detecta)
pd.Series([-1]).isnull()

0    False
dtype: bool

### CARGAMOS NUESTROS DATA SETS

In [32]:
diabetes_df = pd.read_csv("datasets/pima-indians-diabetes.csv", header=None)
diabetes_df

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [33]:
#Cambiamos el nombre de las columnas
diabetes_df.columns = ["pregnancies",
                   "glucose",
                   "blood_pressure",
                   "skin_thickness",
                   "insulin",
                   "bmi",
                   "diabetes_pedigree_function",
                   "age",
                   "outcome"]

diabetes_df.head()

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### CARGAR ARCHIVOS CON DISTINTAS EXTENSIONES

In [34]:
# 1.Creamos unidades de información de los conjuntos de datos
base_url = "https://github.com/njtierney/naniar/raw/master/data/"
datasets_names = ("oceanbuoys", "pedestrian", "riskfactors")
extension = ".rda"

In [35]:
# 2.Descargar y cargar los conjuntos de datos
datasets_dfs = {}

for dataset_name in datasets_names:

    dataset_file = f"{ dataset_name }{ extension }"
    dataset_output_file = f"./datasets/{ dataset_file }"
    dataset_url = f"{ base_url }{ dataset_file }"
    
    !wget -q -O { dataset_output_file } { dataset_url }

    datasets_dfs[f"{ dataset_name }_df"] = pyreadr.read_r(dataset_output_file).get(dataset_name)

datasets_dfs.keys()

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


dict_keys(['oceanbuoys_df', 'pedestrian_df', 'riskfactors_df'])

In [36]:
# 3.Incluir conjuntos de datos en nuestro ambiente local
locals().update(**datasets_dfs)
del datasets_dfs

In [37]:
#4. Verificamos la carga
oceanbuoys_df.shape, pedestrian_df.shape, riskfactors_df.shape, diabetes_df.shape

((736, 8), (37700, 9), (245, 34), (768, 9))

In [38]:
riskfactors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   state             245 non-null    category
 1   sex               245 non-null    category
 2   age               245 non-null    int32   
 3   weight_lbs        235 non-null    object  
 4   height_inch       243 non-null    object  
 5   bmi               234 non-null    float64 
 6   marital           244 non-null    category
 7   pregnant          30 non-null     category
 8   children          245 non-null    int32   
 9   education         244 non-null    category
 10  employment        245 non-null    category
 11  income            245 non-null    category
 12  veteran           242 non-null    category
 13  hispanic          243 non-null    category
 14  health_general    245 non-null    category
 15  health_physical   245 non-null    int32   
 16  health_mental     245 non-

### IMPORTAMOS LA API MEJORADA DE PANDAS

In [39]:
%run pandas-missing-extension.ipynb